In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os
import h5py
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class RedNeuronalGeneral(nn.Module):
    def __init__(self, input_shape, num_classes=3):
        super(RedNeuronalGeneral, self).__init__()
        
        # 1. Calculamos dinámicamente el tamaño aplanado (Flatten)
        self.input_dim = np.prod(input_shape)
        
        print(f"Inicializando red para entrada {input_shape} -> {self.input_dim} neuronas de entrada.")

        # 2. Definimos la arquitectura Flatten + MLP
        #       Flatten -> Dense -> Dense -> Out
        
        self.flatten = nn.Flatten()
        
        self.layers = nn.Sequential(
            # Bloque Denso 1 
            nn.Linear(self.input_dim, 256), # De n entradas a 256 neuronas
            nn.BatchNorm1d(256), # Batch Normalization
            nn.LeakyReLU(0.1), # Activación LeakyReLU alpha=0.1
            nn.Dropout(0.3), # Dropout 30%
            
            # Bloque Denso 2
            nn.Linear(256, 128), # De 256 a 128 neuronas
            nn.BatchNorm1d(128), # Batch Normalization
            nn.LeakyReLU(0.1), # Activación LeakyReLU alpha=0.1
            nn.Dropout(0.2), # Dropout 20%
            
            # Capa de Salida (3 clases: Meningioma, Glioma, T. Pituitario)
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits

In [ ]:

# 2. CONFIGURACIÓN DEL ENTRENAMIENTO
def train_model(model, train_loader, val_loader, epochs, device):
    # Hiperparámetros del documento de Khan
    criterion = nn.CrossEntropyLoss() 
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    model.to(device)
    
    print(f"--- Iniciando entrenamiento en {device} ---")
    
    for epoch in range(epochs):
        # --- FASE DE ENTRENAMIENTO ---
        model.train() # Activa Dropout y BatchNorm
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # a) Resetear gradientes
            optimizer.zero_grad()
            
            # b) Forward Pass
            outputs = model(inputs) # Logits
            
            # c) Calcular Loss
            loss = criterion(outputs, labels)
            
            # d) Backward Pass & Update
            loss.backward()
            optimizer.step()
            
            # Métricas rápidas
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        train_acc = 100 * correct / total
        
        # --- FASE DE VALIDACIÓN ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad(): 
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = 100 * val_correct / val_total
        
        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Acc: {val_acc:.2f}%")

In [ ]:
class BrainTumorDataset(Dataset): #! Eliminar y cambiar a convolución cuántica
    def __init__(self, root_dir, target_size=(64, 64)):
        """
        Args:
            root_dir (string): Directorio con todos los archivos .mat
            target_size (tuple): Tamaño final deseado (64, 64)
        """
        self.root_dir = root_dir
        self.target_size = target_size
        self.files = [f for f in os.listdir(root_dir) if f.endswith('.mat')]
        
        print(f"Dataset cargado: {len(self.files)} imágenes encontradas en {root_dir}")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.root_dir, self.files[idx])
        
        # 1. Cargar el archivo .mat usando h5py
        try:
            with h5py.File(file_path, 'r') as f:

                image_data = np.array(f['cjdata']['image'])
                
                label_data = np.array(f['cjdata']['label'])[0][0]
                
        except Exception as e:
            print(f"Error leyendo {file_path}: {e}")
            return None
        # 2. Preprocesamiento Express
        
        # a) Normalización Min-Max [0, 1] 
        img_min = image_data.min()
        img_max = image_data.max()
        if img_max - img_min != 0:
            image_data = (image_data - img_min) / (img_max - img_min)
        else:
            image_data = np.zeros(image_data.shape)

        # b) Redimensionar (De 512x512 a 64x64)
        # Usamos cv2 por velocidad y eficiencia
        image_resized = cv2.resize(image_data, self.target_size, interpolation=cv2.INTER_AREA)

        # c) Convertir a Tensor de PyTorch y agregar dimensión de canal
        image_tensor = torch.tensor(image_resized, dtype=torch.float32).unsqueeze(0)

        # d) Ajustar Etiqueta
        label_tensor = torch.tensor(int(label_data) - 1, dtype=torch.long)

        return image_tensor, label_tensor


In [ ]:

# --- BLOQUE DE PRUEBA ---
RUTA_DATOS = "C:\\Users\\yleob\\OneDrive - Instituto Politecnico Nacional\\Documents\\1. Leo\\ESCOM\\Semestre 8\\TrabajoTerminal\\Reconocimiento-de-Tumores-Cerebrales\\Proyecto\\DataSet\\Tumores" 

# Verifica primero si existe la carpeta para que no truene el ejemplo
if os.path.exists(RUTA_DATOS):
    # Instanciamos el Dataset
    dataset_real = BrainTumorDataset(root_dir=RUTA_DATOS, target_size=(64, 64))

    # Creamos el DataLoader
    dataloader_real = DataLoader(dataset_real, batch_size=32, shuffle=True)

    # Probamos sacar un lote
    imagenes, etiquetas = next(iter(dataloader_real))

    print(f"\n--- Verificación de Dimensiones ---")
    print(f"Tensor de Imágenes Batch: {imagenes.shape}") 
    # Debería salir: torch.Size([32, 1, 64, 64])
    
    print(f"Tensor de Etiquetas Batch: {etiquetas.shape}")
    # Debería salir: torch.Size([32])

else:
    print(f"No se encontro la carpeta: '{RUTA_DATOS}'")


Dataset cargado: 3064 imágenes encontradas en C:\Users\yleob\OneDrive - Instituto Politecnico Nacional\Documents\1. Leo\ESCOM\Semestre 8\TrabajoTerminal\Reconocimiento-de-Tumores-Cerebrales\Proyecto\DataSet\Tumores

--- Verificación de Dimensiones ---
Tensor de Imágenes Batch: torch.Size([32, 1, 64, 64])
Tensor de Etiquetas Batch: torch.Size([32])
Ejemplo de etiqueta ajustada: 1 (Original era 2)


In [ ]:
from torch.utils.data import random_split, DataLoader
import torch

# 1. Definir los tamaños exactos
total_size = len(dataset_real)
train_size = int(0.70 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size  # El resto para test (para evitar errores de redondeo)

print(f"Total imágenes: {total_size}")
print(f"Entrenamiento (70%): {train_size}")
print(f"Validación (15%): {val_size}")
print(f"Prueba (15%): {test_size}")

# 2. Fijar semilla para reproducibilidad (CRÍTICO PARA TESIS)
# Esto asegura que siempre que se corra el código, el split sea el mismo
generator = torch.Generator().manual_seed(42)

# 3. Realizar el split aleatorio
train_dataset, val_dataset, test_dataset = random_split(
    dataset_real, 
    [train_size, val_size, test_size],
    generator=generator
)

# 4. Crear los DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Total imágenes: 3064
Entrenamiento (70%): 2144
Validación (15%): 459
Prueba (15%): 461
------------------------------
Lotes en Train Loader: 67
Lotes en Val Loader: 15
Lotes en Test Loader: 15


In [ ]:
# 3. SIMULACIÓN DE EXPERIMENTO
# Definimos el dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

input_shape = (1, 64, 64) # Cambiar de acuerdo al objetivo del experimento
modelo = RedNeuronalGeneral(input_shape=input_shape, num_classes=3)

# Pasamos train_loader y val_loader. El test_loader lo guardamos para el final.
train_model(modelo, train_loader, val_loader, epochs=10, device=device)

Usando dispositivo: cuda
Inicializando red para entrada (1, 64, 64) -> 4096 neuronas de entrada.
--- Iniciando entrenamiento en cuda ---
Epoch [1/10] Train Loss: 0.5094 | Train Acc: 77.33% | Val Acc: 84.75%
Epoch [2/10] Train Loss: 0.3207 | Train Acc: 87.31% | Val Acc: 85.40%
Epoch [3/10] Train Loss: 0.2514 | Train Acc: 90.62% | Val Acc: 87.15%
Epoch [4/10] Train Loss: 0.1835 | Train Acc: 93.94% | Val Acc: 80.39%
Epoch [5/10] Train Loss: 0.1652 | Train Acc: 93.66% | Val Acc: 79.96%
Epoch [6/10] Train Loss: 0.1472 | Train Acc: 94.78% | Val Acc: 89.76%
Epoch [7/10] Train Loss: 0.1046 | Train Acc: 96.64% | Val Acc: 82.57%
Epoch [8/10] Train Loss: 0.0903 | Train Acc: 96.60% | Val Acc: 81.05%
Epoch [9/10] Train Loss: 0.0830 | Train Acc: 97.67% | Val Acc: 89.54%
Epoch [10/10] Train Loss: 0.0750 | Train Acc: 97.43% | Val Acc: 82.57%
